In [ ]:
mothur_path = 'C:\\Users\\Haley\\Documents\\MiSeq\\MiSeq_SOP\\mothur\\mothur\\mothur.exe'
inputDir = 'C:\\Users\\Haley\\Documents\\MiSeq\\MiSeq_SOP\\mothur\\mothur\\Data\\Murdochscorpmicrobiome1-90572701\\H5arizonensisscorptelson-167465308'
outputDir = 'C:\\Users\\Haley\\Documents\\MiSeq\\MiSeq_SOP\\test'
processors = 4

In [ ]:
from mothur_py import Mothur
m = Mothur(mothur_path=mothur_path)
m.verbosity=1

# Set directories

In [ ]:
m.set.dir(input=inputDir, output = outputDir)

In [ ]:
m.make.file(type='fastq')

# Make and summarize contigs

In [ ]:
m.make.contigs(file='stability.files', processors = processors)

In [ ]:
m.summary.seqs(fasta='current')

In [ ]:
with open(outputDir + '\\stability.trim.contigs.summary') as f :
    lines = [l.strip() for l in f]
lines = lines[1:]


In [ ]:
lengths = [s.split('\t')[3] for s in lines]
lengths = [int(i) for i in lengths]
long = 0
for i in lengths:
    if i >= 275:
        long += 1
print("Percent of all sequences with length > 275 bp: " + str(round((long/len(lengths)*100),2)) + '%')

In [ ]:
ambigs = [s.split('\t')[4] for s in lines]
ambigs = [int(i) for i in ambigs]
num = 0
for i in ambigs:
    if i > 0:
        num += 1
print("Percent of all sequences with atleast 1 ambiguous base call: " + str(round((num/len(ambigs)*100),2)) + '%')

In [ ]:
# import pandas as pd
# # create pandas data frame; each column is specified as a python dictionary
# x = [1,2,3,4,30]
# df = pd.DataFrame({'lengths': x})
# print(df.head())

# #boxplot example 
# bp = df.boxplot(column = 'lengths')
# bp.set_title("Boxplot of lengths")
# bp.set_xlabel("")
# bp.set_ylabel("read length")

# # barplot example
# bp = df.plot.bar(y = 'lengths', legend = False)
# bp.set_title("Read lengths")
# bp.set_ylabel("length")
# bp.set_xlabel("read #")

## Set Parameters For Screening Step

In [ ]:
maxambig=0
maxlength=275

In [ ]:
m.screen.seqs(fasta='current',group = 'current', maxambig=maxambig, maxlength=maxlength)

## Merge Duplicate Sequences

In [ ]:
m.unique.seqs(fasta='current')

In [ ]:
m.count.seqs(name='current', group='current')

In [ ]:
with open(outputDir + '\\stability.trim.contigs.good.count_table') as f:
    lines = [l.strip() for l in f]
lines = lines[1:]

seqs = [s.split('\t')[1] for s in lines]
seqs = [int(i) for i in seqs]

uniques = len(lines)
total = sum(seqs)

print(str(uniques) + " out of " + str(total) + " sequences are unique.")

## Align Sequences to Reference Alignment

In [ ]:
m.pcr.seqs(fasta='silva.bacteria.fasta', start=11894, end=25319, keepdots='F')

In [ ]:
m.align.seqs(fasta='stability.trim.contigs.good.unique.fasta', reference= 'silva.bacteria.pcr.fasta')

## Re-screen to Remove Sequences With Indels and Homopolymers Longer than 8 Bases

In [ ]:
m.summary.seqs(fasta='current', count = 'current')

In [ ]:
m.screen.seqs(fasta='current', count='current', summary='current', 
start=1968, end=11550, maxhomop=8)

# Filter sequences to remove columns that only contain gaps and extra gaps at ends

In [ ]:
m.filter.seqs(fasta='current', vertical='T', trump='.')

## Merge any new duplicate sequences

In [ ]:
m.unique.seqs(fasta='current', count='current')

## Pre-cluster (merge) sequences within 2 bases of each other

In [ ]:
m.pre.cluster(fasta='current', count = 'current', diffs = 2)

In [ ]:
with open(outputDir + '\\stability.trim.contigs.good.unique.good.filter.unique.precluster.count_table') as f:
    lines = [l.strip() for l in f]
lines = lines[1:]

seqs = [s.split('\t')[1] for s in lines]
seqs = [int(i) for i in seqs]

uniques = len(lines)
total = sum(seqs)

print(str(uniques) + " out of " + str(total) + " sequences are unique.")

## Search for and remove chimeric sequences

In [ ]:
m.chimera.uchime(fasta = 'stability.trim.contigs.good.unique.good.filter.unique.precluster.fasta', count = 'stability.trim.contigs.good.unique.good.filter.unique.precluster.count_table', dereplicate = 't')

In [ ]:
m.remove.seqs(fasta='current', accnos='current', count='current')

# Classify sequences based on taxonomy to remove undesirables 

In [ ]:
m.classify.seqs(fasta='current', count='current', reference='trainset9_032012.pds.fasta', taxonomy='trainset9_032012.pds.tax', cutoff=80)

In [ ]:
m.remove.lineage(fasta='current', count='current', taxonomy='current', taxon='Chloroplast-Mitochondria-unknown-Archaea-Eukaryota')

In [ ]:
m.summary.tax(taxonomy='current', count='current')

## Cluster sequences and generate distance matrix and OTUs

In [ ]:
m.cluster.split(fasta='stability.trim.contigs.good.unique.good.filter.unique.precluster.pick.pick.fasta', count='stability.trim.contigs.good.unique.good.filter.unique.precluster.denovo.uchime.pick.pick.pick.count_table', taxonomy='stability.trim.contigs.good.unique.good.filter.unique.precluster.pick.pds.wang.pick.taxonomy', splitmethod='classify', taxlevel=4, cutoff=0.03)

## Get consensus tanomy of OTUs

In [ ]:
m.classify.otu(list='current', count='current', taxonomy='current')